### Training
それでは学習を始めましょう。まず、XGBoost のコンテナの場所を取得します。コンテナ自体は SageMaker 側で用意されているので、場所を指定すれば利用可能です。

まずは、データの前処理でS3に保存したファイルのパスを取得します。

In [ ]:
!aws s3 ls s3://###sagemaker default bucket###/xgboost-churn-stepfunctions/xgboost-churn

In [ ]:
import sagemaker
from sagemaker import get_execution_role
role = get_execution_role()
sess = sagemaker.Session()
bucket = sess.default_bucket()
prefix = 'xgboost-churn-stepfunctions/xgboost-churn'
sagemaker.__version__

上記セルを実行して、SageMaker Python SDK Version が 1.xx.x の場合、以下のセルのコメントアウトを解除してから実行してください。実行が完了したら、上にあるメニューから [Kernel] -> [Restart kernel] を選択してカーネルを再起動してください。

再起動が完了したら、このノートブックの一番上のセルから再度実行してください。その場合、以下のセルを実行する必要はありません。

In [ ]:
# !pip install -U --quiet "sagemaker==2.16.1"

In [ ]:
# 前処理データをダウンロード
# s3 = boto3.resource('s3')
# s3.Bucket(bucket).download_file('{}/{}'.format(prefix, 'train.csv'), 'train.csv')
# s3.Bucket(bucket).download_file('{}/{}'.format(prefix, 'validation.csv'), 'validation.csv')
# s3.Bucket(bucket).download_file('{}/{}'.format(prefix, 'test.csv'), 'test.csv')

開発時に学習で利用する場所にデータをアップロードします。

In [ ]:
# 学習用データとしてアップロード
input_train = sess.upload_data(path='train.csv', key_prefix='xgboost-churn-stepfunctions/xgboost-churn-input')
input_validation = sess.upload_data(path='validation.csv', key_prefix='xgboost-churn-stepfunctions/xgboost-churn-input')
input_test = sess.upload_data(path='validation.csv', key_prefix='xgboost-churn-stepfunctions/xgboost-churn-input')

In [ ]:
input_train

In [ ]:
# from sagemaker.session import s3_input
from sagemaker.inputs import TrainingInput

input_train_prefix = 's3://{}/{}/train'.format(bucket, 'xgboost-churn-stepfunctions/xgboost-churn-input')
input_validation_prefix = 's3://{}/{}/validation'.format(bucket, 'xgboost-churn-stepfunctions/xgboost-churn-input')

content_type='text/csv'
s3_input_train = TrainingInput(input_train_prefix, content_type=content_type)
s3_input_validation = TrainingInput(input_validation_prefix, content_type=content_type)

### 学習の実行

In [ ]:
import boto3
container = sagemaker.image_uris.retrieve("xgboost", boto3.Session().region_name, "1.2-1")

In [ ]:
s3_output_location = 's3://{}/{}/output'.format(bucket, prefix)
print(s3_output_location)

学習のためにハイパーパラメータを指定したり、学習のインスタンスの数やタイプを指定することができます。XGBoost における主要なハイパーパラメータは以下のとおりです。

- max_depth アルゴリズムが構築する木の深さをコントロールします。深い木はより学習データに適合しますが、計算も多く必要で、overfiting になる可能性があります。たくさんの浅い木を利用するか、少数の深い木を利用するか、モデルの性能という面ではトレードオフがあります。
- subsample 学習データのサンプリングをコントロールします。これは overfitting のリスクを減らしますが、小さすぎるとモデルのデータが不足してしまいます。
- num_round ブースティングを行う回数をコントロールします。以前のイテレーションで学習したときの残差を、以降のモデルにどこまで利用するかどうかを決定します。多くの回数を指定すると学習データに適合しますが、計算も多く必要で、overfiting になる可能性があります。
- eta 各ブースティングの影響の大きさを表します。大きい値は保守的なブースティングを行います。
- gamma ツリーの成長の度合いをコントロールします。大きい値はより保守的なモデルを生成します。

XGBoostのhyperparameterに関する詳細は github もチェックしてください。

In [ ]:
hyperparameters = {"max_depth":"5",
                        "eta":"0.2",
                        "gamma":"4",
                        "min_child_weight":"6",
                        "subsample":"0.8",
                        "objective":"binary:logistic",
                        "num_round":"100"}

xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    hyperparameters=hyperparameters,
                                    instance_count=1, 
                                    instance_type='ml.m4.xlarge',
                                    sagemaker_session=sess
                                   )

In [ ]:
xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})

### Evaluation

#### SageMaker Endpointを利用して評価

In [ ]:
xgb_predictor = xgb.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

現在、エンドポイントをホストしている状態で、これを利用して簡単に予測を行うことができます。予測は http の POST の request を送るだけです。 ここではデータを numpy の array の形式で送って、予測を得られるようにしたいと思います。しかし、endpoint は numpy の array を受け取ることはできません。

このために、csv_serializer を利用して、csv 形式に変換して送ることができます。

In [ ]:
xgb_predictor.serializer = sagemaker.serializers.CSVSerializer()

In [ ]:
import pandas as pd
import numpy as np

def predict(data, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, xgb_predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

test_data = pd.read_csv('test.csv')
dtest = test_data.values
predictions = []
predictions.append(predict(dtest[:, 1:]))
predictions = np.array(predictions).squeeze()

機械学習の性能を比較評価する方法はいくつかありますが、単純に、予測値と実際の値を比較しましょう。今回は、顧客が離反する 1 と離反しない 0 を予測しますので、この混同行列を作成します。

In [ ]:
pd.crosstab(index=test_data.iloc[:, 0], columns=np.round(predictions), rownames=['actual'], colnames=['predictions'])

※ 注意点, アルゴリズムにはランダムな要素があるので結果は必ずしも一致しません.

48人の離反者がいて、それらの39名 (true positives) を正しく予測できました。そして、4名の顧客は離反すると予測しましたが、離反していません (false positives)。9名の顧客は離反しないと予測したにもかかわらず離反してしまいました (false negatives)。

重要な点として、離反するかどうかを np.round() という関数で、しきい値0.5で判断しています。xgboost が出力する値は0から1までの連続値で、それらを離反する 1 と 離反しない 0 に分類します。しかし、その連続値 (離反する確率) が示すよりも、顧客の離反というのは損害の大きい問題です。つまり離反する確率が低い顧客も、しきい値を0.5から下げて、離反するとみなす必要があるかもしれません。もちろんこては、false positives （離反すると予測したけど離反しなかった）を増やすと思いますが、 true positives (離反すると予測して離反した) を増やし、false negatives (離反しないと予測して離反した）を減らせます。

直感的な理解のため、予測結果の連続値をみてみましょう。

In [ ]:
import matplotlib.pyplot as plt
plt.hist(predictions)
plt.show()

連続値は0から1まで歪んでいますが、0.1から0.9までの間で、しきい値を調整するにはちょうど良さそうです。


例えば、しきい値を0.5から0.3まで減らしてみたとき、true positives は 1 つ、false positives は 3 つ増え、false negatives は 1 つ減りました。全体からみると小さな値ですが、全体の6-10%の顧客が、しきい値の変更で、予測結果が変わりました。ここで5名にインセンティブを与えることによって、インセンティブのコストが掛かりますが、3名の顧客を引き止めることができるかもしれません。 つまり、最適な閾値を決めることは、実世界の問題を機械学習で解く上で重要なのです。これについてもう少し広く議論し、仮説的なソリューションを考えたいと思います。

#### 推論エラーのコストをビジネスモデルから定義

2値分類の問題においては、しきい値に注意しなければならないという、似たような状況に直面することが多いです。それ自体は問題ではありません。もし、出力の連続値が2クラスで完全に別れていれば、MLを使うことなく単純なルールで解くことができると考えられます。

重要なこととして、MLモデルを正版環境に導入する際、モデルが false positives と false negatives に誤って入れたときのコストがあげられます。しきい値の選択は4つの指標に影響を与えます。4つの指標に対して、ビジネス上の相対的なコストを考える必要があるでしょう。



携帯電話会社の離反の問題において、コストとはなんでしょうか？コストはビジネスでとるべきアクションに結びついています。いくつかの仮定をおいてみましょう。

まず、true negatives のコストとして 0USD を割り当てます。満足しているお客様を正しく認識できていれば何も実施しません。

false negatives が一番問題で、なぜなら、離反していく顧客を正しく予測できないからです。顧客を失えば、再獲得するまでに多くのコストを払う必要もあり、例えば逸失利益、広告コスト、管理コスト、販売管理コスト、電話の購入補助金などがあります。インターネットを簡単に検索してみると、そのようなコストは数百ドルとも言われ、ここでは 500USD としましょう。これが false negatives に対するコストです。

最後に、離反していくと予測された顧客に 100USD のインセンティブを与えることを考えましょう。 携帯電話会社がそういったインセンティブを提供するなら、2回くらいは離反の前に考え直すかもしれません。これは true positive と　false negative のコストになります。false positives の場合 (顧客は満足していて、モデルが誤って離反しそうと予測した場合)、 100USD のインセンティブは捨てることになります。その 100USD を効率よく消費してしまうかもしれませんが、優良顧客へのロイヤリティを増やすという意味では悪くないかもしれません。

#### コストの計算式

alse negatives が false positives よりもコストが高いことは説明しました。そこで、顧客の数ではなく、コストを最小化するように、しきい値を最適化することを考えましょう。コストの関数は以下のようなものになります。

txt
500USD * FN(C) + 0USD * TN(C) + 100USD * FP(C) + 100USD * TP(C)
FN(C) は false negative の割合で、しきい値Cの関数です。同様にTN, FP, TP も用意します。この関数の値が最小となるようなしきい値Cを探します。 最も単純な方法は、候補となる閾値で何度もシミュレーションをすることです。以下では100個の値に対してループで計算を行います。

In [ ]:
cutoffs = np.arange(0.01, 1, 0.01)
costs = []

for c in cutoffs:
    _predictions = pd.Categorical(np.where(predictions > c, 1, 0), categories=[0, 1])
    matrix_a = np.array([[0, 100], [500, 100]])
    matrix_b = pd.crosstab(index=test_data.iloc[:, 0], columns=_predictions, dropna=False)
    costs.append(np.sum(np.sum(matrix_a * matrix_b)))

costs = np.array(costs)
plt.plot(cutoffs, costs)
plt.show()
print('Cost is minimized near a cutoff of:', cutoffs[np.argmin(costs)], 'for a cost of:', np.min(costs))

#### エンドポイント の削除

In [ ]:
xgb_predictor.delete_endpoint()

#### マニュアルで評価

In [ ]:
!pip install xgboost

In [ ]:
model_path = xgb.model_data
print(model_path)

In [ ]:
sagemaker.s3.S3Downloader.download(model_path, './')

In [ ]:
!tar xvzf model.tar.gz

In [ ]:
import pickle
model = pickle.load(open('xgboost-model', 'rb'))

In [ ]:
import xgboost
xgboost.plot_importance(model)

In [ ]:
test_dm = xgboost.DMatrix(test_data.values[:, 1:])
predictions_xgb = model.predict(test_dm)

In [ ]:
pd.crosstab(index=test_data.iloc[:, 0], columns=np.round(predictions_xgb), rownames=['actual'], colnames=['predictions'])

In [ ]:
from sklearn import metrics
metrics.accuracy_score(test_data.iloc[:, 0].values, np.round(predictions_xgb))

#### evaluationスクリプトの作成

### Evalutaion jobの作成

#### processing docker imageの作成

In [ ]:
import boto3
# boto3の機能を使ってリポジトリ名に必要な情報を取得する
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name
print(region)
print(account_id)
ecr_repository = 'xgboost-churn-evaluation'
tag = ':latest'
repository_uri = '{}.dkr.ecr.{}.amazonaws.com/{}'.format(account_id, region, ecr_repository + tag)

!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
# リポジトリの作成
# すでにある場合はこのコマンドは必要ありません。
!aws ecr create-repository --repository-name $ecr_repository

In [ ]:
!docker build -f Dockerfile-evaluation -t xgboost-churn-evaluation .

In [ ]:
# docker imageをecrにpush
!docker tag {ecr_repository + tag} $repository_uri
!docker push $repository_uri

#### local からprocessingを実行

In [ ]:
from sagemaker import get_execution_role
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput
role = get_execution_role()

script_processor = ScriptProcessor(
    image_uri='%s.dkr.ecr.ap-northeast-1.amazonaws.com/%s:latest' % (account_id, ecr_repository),
    role=role,
    command=['python3'],
    instance_count=1,
    instance_type='ml.m5.xlarge')

In [ ]:
model_data_s3_uri = model_path

script_processor.run(code='evaluation.py',
    inputs=[ProcessingInput(
            source='test.csv',
            destination='/opt/ml/processing/input',
            input_name='input-1'),
            ProcessingInput(
            source=model_data_s3_uri,
            destination='/opt/ml/processing/model',
            input_name='input-2')],
)

ここまでで、SageMaker環境でコンテナを活用した前処理、学習、検証が実行できました。